# Custom derived quantities
This notebook demonstrates how to extent MIKE IO 1D with your own reuseable derived quantity.

#### 1 - Exploratory design of the derived quantity

In [ ]:
# Let's open a Res1D file.
from mikeio1d import Res1D
res = Res1D("../tests/testdata/network.res1d")
res

In [ ]:
# We will make a derived quantity that detects instabilities in discharge. Let's first read the discharge at a sample point.
res.nodes['1'].WaterLevel.plot()

In [ ]:
# We'll use a simple approach of taking the rolling standard deviation as an indicator of instability.
df = res.nodes['1'].WaterLevel.read()
df.rolling(3).std().plot()

In [ ]:
# Let's find the node with the highest instability per out definition.
(
    res.nodes.WaterLevel.read()
    .rolling(3)
    .std()
    .max()
    .idxmax()
)

In [ ]:
# Let's plot the node found to have the highest instability. Indeed, we found something fishy.
res.nodes['Weir Outlet:119w1'].WaterLevel.plot()

#### 2 - Create a reuseable derived quantity

In [ ]:
# We'll need some imports
from mikeio1d.quantities import DerivedQuantity
from mikeio1d.quantities import TimeSeriesIdGroup

In [ ]:
# Now let's build our derived quantity. For that, we need to create a class that inherits from DerivedQuantity.

class NodeWaterLevelInstability(DerivedQuantity):
    # This is the name of our derived quantity
    _NAME = "NodeWaterLevelInstability"
    # These are the groups the quantity belongs to. Since we're looking at nodes, we put it in the NODE group.
    _GROUPS = {TimeSeriesIdGroup.NODE}
    # This is the actual quantity we're deriving from. In this case, it's the water level.
    _SOURCE_QUANTITY = "WaterLevel"

    # This contains the logic we created in the last step.
    def derive(self, df_source, locations):
        return df_source.rolling(3).std()

In [ ]:
# We need to tell MIKE IO 1D that we have a new derived quantity.
from mikeio1d.res1d import derived_quantity_manager as dqm
dqm.register(NodeWaterLevelInstability)

#### 3 - Test our new derived quantity

In [ ]:
# We need to reload our result file.
res = Res1D("../tests/testdata/network.res1d")
res

In [ ]:
# Let's check out the node we know to be instable. Notice our new derived quantity is listed in the repr below.
instable_node = res.nodes['Weir Outlet:119w1']
instable_node

In [ ]:
# Let's plot the derived quantity. It looks reasonable.
instable_node.NodeWaterLevelInstability.plot(legend=False)

In [ ]:
# Let's read the derived quantity for all nodes.
res.nodes.NodeWaterLevelInstability.read().head(5)